In [25]:
from google.colab import files
up = files.upload()
csv_path = './dataset_sujo.csv'
import pandas as pd, numpy as np, unicodedata
df = pd.read_csv(csv_path)

Saving dataset_sujo.csv to dataset_sujo (1).csv


In [26]:
print(df.shape)
print(df.dtypes)


(5075, 8)
 Nome             object
Idade            float64
CIDADE            object
Data Admissão     object
salario (R$)     float64
Estado-civil      object
Cargo             object
Telefone          object
dtype: object


In [27]:
def normalize_col(c):
    c = unicodedata.normalize("NFKD", str(c)).encode("ASCII","ignore").decode("ASCII")
    c = c.strip().lower().replace(" ","_")
    c = "".join(ch if (ch.isalnum() or ch=="_") else "_" for ch in c)
    while "__" in c: c = c.replace("__","_")
    return c.strip("_")
df.columns = [normalize_col(c) for c in df.columns]
df.columns


Index(['nome', 'idade', 'cidade', 'data_admissao', 'salario_r', 'estado_civil',
       'cargo', 'telefone'],
      dtype='object')

- Padronização dos nomes das colunas

In [28]:
date_cols = [c for c in df.columns if "data" in c or "date" in c]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce", dayfirst=True)
for c in ["idade","salario"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
df.dtypes


,0
nome,object
idade,float64
cidade,object
data_admissao,datetime64[ns]
salario_r,float64
estado_civil,object
cargo,object
telefone,object


- Tipos corretos para cada etapa

In [30]:
def clean_str(s):
    if pd.isna(s): return s
    s = str(s).strip()
    s = " ".join(s.split())
    s = unicodedata.normalize("NFKD", s).encode("ASCII","ignore").decode("ASCII")
    return s.title()
for c in ["nome","cidade","estado_civil","cargo"]:
    if c in df.columns:
        df[c] = df[c].map(clean_str)
df.head(3)

null_ratio = df.isna().mean()
drop_cols = null_ratio[null_ratio>0.9].index.tolist()
df = df.drop(columns=drop_cols)
drop_cols


['telefone']

- removendo colunas inúteis ou vazias

In [31]:
if "idade" in df.columns:
    df.loc[(df["idade"]<14) | (df["idade"]>100), "idade"] = pd.NA
if "data_admissao" in df.columns:
    now = pd.Timestamp("today").normalize()
    df.loc[df["data_admissao"]>now, "data_admissao"] = pd.NaT
df.head(3)


,nome,idade,cidade,data_admissao,salario_r,estado_civil,cargo
0,Saulo Souza,37.0,Curitiba,2023-10-24,3635.86,Solteiro,Analista
1,Paula Pereira,39.0,Rio De Janeiro,2024-06-04,3089.28,Casado,Engenheiro De Dados
2,Gabi Oliveira,16.0,Curitiba,2025-01-09,3378.62,Solteiro,Analista


- Inconsistência de dados impossíveis com idade e datas